# Machine Learning Foundation

## Course 4, Part e: Non-Negative Matrix Factorization DEMO

This exercise illustrates usage of Non-negative Matrix factorization and covers techniques related to sparse matrices and some basic work with Natural Langauge Processing.  We will use NMF to look at the top words for given topics.

## Data

We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

These data are available in the data folder (or online [here](http://mlg.ucd.ie/files/datasets/bbc.zip)). The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix (a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html) format can be useful for matrices with many values that are 0, and save space by storing the position and values of non-zero elements).
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.

## Data Setup

In [1]:
import os
os.chdir('/users/salahkaf/desktop/Data')

In [2]:
with open('bbc.mtx') as f:
    content = f.readlines()

In [3]:
# Remove first two elements
content.pop(0)
content.pop(0)

'9635 2225 286774\n'

## Part 1

Here, we will turn this into a list of tuples representing a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html). Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of our list will be:

`(1, 3, 2)`

In [4]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
# Let's examine the first few elements
sparsemat[:8]

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1)]

## Part 2: Preparing Sparse Matrix data for NMF 

We will use the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) function to turn the sparse matrix into an array. 

In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix  # Sparse matrix are usually used in NLP
cols = [x[0]-1 for x in sparsemat]
rows = [x[1]-1 for x in sparsemat]
values = [x[2] for x in sparsemat]
coo = coo_matrix((values, (rows, cols)))

In [6]:
with open('bbc.terms')as f:
    content = f.readlines()
words = [c.split()[0] for c in content]
words[:8] # words 

['ad', 'sale', 'boost', 'time', 'warner', 'profit', 'quarterli', 'media']

In [7]:
with open('bbc.docs') as f:
    content = f.readlines()
docs = [c.split()[0] for c in content]
docs[:8] # Docs real name 

['business.001',
 'business.002',
 'business.003',
 'business.004',
 'business.005',
 'business.006',
 'business.007',
 'business.008']

In [8]:
pd.DataFrame(coo.toarray(), columns = words, index= docs)

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
business.001,1,5,2,3,4,10,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
business.002,0,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
business.003,0,4,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
business.004,0,1,0,0,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
business.005,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tech.397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.398,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.399,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tech.400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## NMF

NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 

Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html).

## Part 3

Here, we will import `NMF`, define a model object with 5 components, and `fit_transform` the data created above.

In [9]:
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=818) # 5 because we have five major topics
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 9636 observations (articles) and five latent features

(2225, 5)

In [10]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

array([1, 1, 1, ..., 3, 3, 2])

## Part 4: 

Check out the `components` of this model:

In [11]:
model.components_.shape

(5, 9635)

In [12]:
topic_word = pd.DataFrame(model.components_.round(3), 
            index = ["topic_1", "topic_2", "topic_3", "topic_4", "topic_5"],
            columns = words)
topic_word

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
topic_1,1.110,0.000,0.148,1.805,0.000,0.000,0.000,0.290,0.000,0.008,...,0.000,0.00,0.004,0.005,0.000,0.000,0.000,0.000,0.000,0.000
topic_2,0.907,2.135,0.556,1.620,0.015,1.233,0.091,0.059,0.347,0.206,...,0.001,0.00,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.001
topic_3,0.678,0.412,0.051,4.003,0.038,0.000,0.000,0.190,0.084,0.128,...,0.001,0.00,0.000,0.000,0.000,0.000,0.000,0.002,0.005,0.000
topic_4,0.696,0.416,0.055,1.220,0.052,0.051,0.008,1.221,0.280,0.000,...,0.025,0.02,0.061,0.075,0.136,0.021,0.040,0.029,0.028,0.020
topic_5,0.475,0.373,0.124,0.986,0.022,0.042,0.000,0.052,0.021,0.252,...,0.000,0.00,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000


The original data had 5 topics, as listed in bbc.docs (which these topic words relate to).

Business

Entertainment

Politics

Sport

Tech

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to get a list of the top 10 words for each topic. We can just store this in a list of lists.

In [13]:
with open('bbc.docs') as f:
    content = f.readlines()
docs = [c.split()[0] for c in content]


topic_doc = pd.DataFrame(doc_topic.round(5), 
            index = [i.split(".")[0] for i in docs],
            columns = ["topic_1", "topic_2", "topic_3", "topic_4", "topic_5"])
topic_doc

,topic_1,topic_2,topic_3,topic_4,topic_5
business,0.00000,0.27757,0.00680,0.08380,0.00000
business,0.03033,0.31001,0.00311,0.00000,0.00000
business,0.03685,0.17981,0.00000,0.00000,0.00000
business,0.00000,0.40197,0.00000,0.00000,0.00982
business,0.00061,0.14514,0.00890,0.03643,0.00765
...,...,...,...,...,...
tech,0.00000,0.10331,0.00000,0.37516,0.00000
tech,0.02170,0.05943,0.00000,0.29394,0.00000
tech,0.25508,0.22328,0.03813,0.26931,0.00000
tech,0.05870,0.09831,0.00509,0.15505,0.00511


In [14]:
topic_doc.reset_index().groupby('index').mean().idxmax()


topic_1         politics
topic_2         business
topic_3            sport
topic_4             tech
topic_5    entertainment
dtype: object

In [15]:
topic_word.T.sort_values(by='topic_1', ascending=False)


,topic_1,topic_2,topic_3,topic_4,topic_5
parti,7.272,0.000,0.000,0.000,0.0
labour,6.860,0.000,0.000,0.000,0.0
govern,6.403,2.868,0.000,0.000,0.0
elect,5.984,0.000,0.000,0.000,0.0
blair,5.031,0.000,0.000,0.000,0.0
...,...,...,...,...,...
tidal,0.000,0.017,0.000,0.000,0.0
serg,0.000,0.015,0.019,0.000,0.0
hanov,0.000,0.004,0.000,0.029,0.0
marin,0.000,0.000,0.022,0.114,0.0


**Note**: Just like we read in the data above, we'll have to read in the words from the bbc.terms file.

---
### Machine Learning Foundation (C) 2020 IBM Corporation